In [2]:
pip install pyspark


Note: you may need to restart the kernel to use updated packages.


In [3]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, IntegerType
from pyspark.sql.functions import col
from pyspark.sql.functions import sum,count
from pyspark.sql.functions import stddev_pop
from pyspark.sql.functions import max
from pyspark.sql.functions import mean ,corr,max, min,stddev_pop ,stddev_samp,covar_pop,covar_samp
spark = SparkSession.builder.getOrCreate()
sc =SparkContext.getOrCreate()




Reading textfile and FLATMAP


In [4]:
my_rdd = sc.textFile("sample.txt")
my_rdd = my_rdd.flatMap(lambda r: r.split(" "))
my_rdd.collect()
#map_result = data.flatmap(lambda r: r[5].split("|"))
#map_result.collect()

['web', 'and', 'data', 'science', 'mathematical', 'modelling']

MAP

In [5]:
data1 = sc.textFile("sample.txt")
map_result1 = data1.map(lambda line:line.split(" "))
map_result1.collect()

[['web', 'and', 'data', 'science'], ['mathematical', 'modelling']]

Adding new word or letter

In [6]:
words = sc.parallelize(
        ["BIG DATA",
         "DATA SCIENCE",
        "OPTIMIZATION",
        "NETWORK THEORY"]
        )

data2= words.map(lambda r: (r,("course")))
data2.collect()

[('BIG DATA', 'course'),
 ('DATA SCIENCE', 'course'),
 ('OPTIMIZATION', 'course'),
 ('NETWORK THEORY', 'course')]

FILTERING

In [7]:
words = sc.parallelize(
        ["BIG DATA",
         "DATA SCIENCE",
        "OPTIMIZATION",
        "NETWORK THEORY"]
        )
data3 = words.filter(lambda r:"DATA" in r)
data3.collect()

['BIG DATA', 'DATA SCIENCE']

READ CSV FILE

In [8]:
#spark = SparkSession.builder.getOrCreate()
df = spark.read.option("inferschema","true").option("header","true").csv("assignment01_flight.csv")
df.show()
#print(df.dtypes)


+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

print(df.schema)

FINDING COUNTS WITH GREATER THAN CONDITION

In [9]:
df.createOrReplaceTempView("flighttable")
spark.sql("""select count(*) from flighttable where count>'10'""").show()

+--------+
|count(1)|
+--------+
|     208|
+--------+



In [10]:
df1=df.where(col("count") > '10')
df1.count()

208

FINDING COUNT


In [11]:
spark.sql("""select SUM(count) from flighttable where DEST_COUNTRY_NAME = 'United States'""").show()

+----------+
|sum(count)|
+----------+
|    411352|
+----------+



In [12]:
df2 = df.where(col("DEST_COUNTRY_NAME") == "United States").agg(sum("count"))
df2.show()

+----------+
|sum(count)|
+----------+
|    411352|
+----------+



In [13]:
df7=df.select(("*")).filter(col("DEST_COUNTRY_NAME") == "United States").groupBy("DEST_COUNTRY_NAME").sum()
df7.show()

+-----------------+----------+
|DEST_COUNTRY_NAME|sum(count)|
+-----------------+----------+
|    United States|    411352|
+-----------------+----------+



In [24]:
df7=df.filter(col("DEST_COUNTRY_NAME") == "United States").groupBy("ORIGIN_COUNTRY_NAME").agg(sum("count")).sort(col("ORIGIN_COUNTRY_NAME"))
df7.explain()

== Physical Plan ==
*(3) Sort [ORIGIN_COUNTRY_NAME#17 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(ORIGIN_COUNTRY_NAME#17 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#243]
   +- *(2) HashAggregate(keys=[ORIGIN_COUNTRY_NAME#17], functions=[sum(cast(count#18 as bigint))])
      +- Exchange hashpartitioning(ORIGIN_COUNTRY_NAME#17, 200), ENSURE_REQUIREMENTS, [id=#239]
         +- *(1) HashAggregate(keys=[ORIGIN_COUNTRY_NAME#17], functions=[partial_sum(cast(count#18 as bigint))])
            +- *(1) Project [ORIGIN_COUNTRY_NAME#17, count#18]
               +- *(1) Filter (isnotnull(DEST_COUNTRY_NAME#16) AND (DEST_COUNTRY_NAME#16 = United States))
                  +- FileScan csv [DEST_COUNTRY_NAME#16,ORIGIN_COUNTRY_NAME#17,count#18] Batched: false, DataFilters: [isnotnull(DEST_COUNTRY_NAME#16), (DEST_COUNTRY_NAME#16 = United States)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/User/assignment01_flight.csv], PartitionFilters: [], PushedFilters: [IsNotNull(DEST_CO

FINDING MAX VALUE

In [15]:
spark.sql("""select max(count) from flighttable """).show()

+----------+
|max(count)|
+----------+
|    370002|
+----------+



In [205]:
df3 = df.select(max("count"))
df3.show()

+----------+
|max(count)|
+----------+
|    370002|
+----------+

